In [20]:
import warnings
import pandas as pd
import numpy as np
import implicit

from src.utils import get_score_top, get_score_model, aggregate_users, create_sparse_matrix

warnings.filterwarnings('ignore')

In [2]:
interactions_df = pd.read_csv('interactions.csv')
interactions_df.head()

,row,col,data
0,0,3568,1.0
1,0,3827,1.0
2,0,4844,1.0
3,0,5734,1.0
4,0,6518,1.0


In [3]:
msk = np.random.rand(len(interactions_df)) < 0.8
train = interactions_df[msk]
test = interactions_df[~msk]

In [4]:
tops = train.groupby('col').count().sort_values('data', ascending=False)
tops

,row,data
col,,
17955,722,722
8638,722,722
5113,691,691
10227,652,652
4657,643,643
...,...,...
15356,1,1
7167,1,1
7163,1,1


In [5]:
top_items = list(np.array(tops.index)[:30])
top_items

[17955,
 8638,
 5113,
 10227,
 4657,
 3922,
 4361,
 8982,
 10466,
 197,
 6289,
 10067,
 2301,
 12469,
 7581,
 4634,
 3806,
 5297,
 3572,
 5645,
 8666,
 15892,
 9461,
 8481,
 3565,
 6404,
 4844,
 10068,
 5562,
 1212]

In [6]:
aggregated_test = aggregate_users(test.values)
aggregated_train = aggregate_users(train.values)

In [21]:
get_score_top(aggregated_test, top_items)

100%|██████████████████████████████████| 20485/20485 [00:00<00:00, 98947.10it/s]


0.00900577313841487

In [8]:
X_sparse = create_sparse_matrix(30910, 18494, aggregated_train)

In [9]:
X_sparse.shape

(30910, 18495)

In [22]:
model = implicit.als.AlternatingLeastSquares(factors=64, regularization=0.0, iterations=16)
model.fit(X_sparse)

  0%|          | 0/16 [00:00<?, ?it/s]

In [23]:
get_score_model(aggregated_test, aggregated_train, model, 18494)

100%|████████████████████████████████████| 20485/20485 [00:30<00:00, 661.84it/s]


0.02356688684996079

In [24]:
model = implicit.nearest_neighbours.CosineRecommender(K=3)
model.fit(X_sparse)

  0%|          | 0/18495 [00:00<?, ?it/s]

In [25]:
get_score_model(aggregated_test, aggregated_train, model, 18494)

100%|███████████████████████████████████| 20485/20485 [00:03<00:00, 5315.01it/s]


0.0268107666656325